In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [4]:
sf_df = pd.read_csv('restaurant.csv')
sf_df.head(5)

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,...,inspection_type,violation_id,violation_description,risk_category,Neighborhoods (old),Police Districts,Supervisor Districts,Fire Prevention Districts,Zip Codes,Analysis Neighborhoods
0,101192,Cochinita #2,2 Marina Blvd Fort Mason,San Francisco,CA,NaN,NaN,NaN,NaN,1.415043e+10,...,New Ownership,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,97975,BREADBELLY,1408 Clement St,San Francisco,CA,94118,NaN,NaN,NaN,1.415724e+10,...,Routine - Unscheduled,97975_20190725_103124,Inadequately cleaned or sanitized food contact...,Moderate Risk,NaN,NaN,NaN,NaN,NaN,NaN
2,92982,Great Gold Restaurant,3161 24th St.,San Francisco,CA,94110,NaN,NaN,NaN,NaN,...,New Ownership,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,101389,HOMAGE,214 CALIFORNIA ST,San Francisco,CA,94111,NaN,NaN,NaN,1.415488e+10,...,New Construction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,85986,Pronto Pizza,798 Eddy St,San Francisco,CA,94109,NaN,NaN,NaN,NaN,...,New Ownership,85986_20161011_103114,High risk vermin infestation,High Risk,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
sf_df.shape

(53973, 23)

In [6]:
sf_df.dropna(subset=['business_id','business_name',
                      'business_address','business_city','business_state',
                      'business_postal_code','business_latitude','business_longitude',
                      'business_location','business_phone_number','inspection_id',
                      'inspection_id','inspection_date','inspection_score','inspection_type',
                      'violation_id','violation_description'],inplace=True)

In [7]:
sf_df.head()

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,...,inspection_type,violation_id,violation_description,risk_category,Neighborhoods (old),Police Districts,Supervisor Districts,Fire Prevention Districts,Zip Codes,Analysis Neighborhoods
11,4794,VICTOR'S,210 TOWNSEND St,San Francisco,CA,94107,37.778634,-122.393089,POINT (-122.393089 37.778634),1.415561e+10,...,Routine - Unscheduled,4794_20181030_103138,Improper storage use or identification of toxi...,Low Risk,34.0,2.0,9.0,6.0,28856.0,34.0
172,63652,SFDH - Banquet Main Kitchen,450 Powell St 2nd Floor,San Francisco,CA,94102,37.788918,-122.408507,POINT (-122.408507 37.788918),1.415540e+10,...,Routine - Unscheduled,63652_20190904_103133,Foods not protected from contamination,Moderate Risk,6.0,1.0,10.0,5.0,28852.0,8.0
327,328,Miyako,1470 Fillmore St,San Francisco,CA,94115,37.783017,-122.432584,POINT (-122.432584 37.783017),1.415554e+10,...,Routine - Unscheduled,328_20161122_103103,High risk food holding temperature,High Risk,41.0,9.0,11.0,15.0,29490.0,39.0
372,2684,ERIC'S RESTAURANT,1500 Church St,San Francisco,CA,94131,37.746759,-122.426995,POINT (-122.426995 37.746759),1.415528e+10,...,Routine - Unscheduled,2684_20190715_103109,Unclean or unsanitary food contact surfaces,High Risk,22.0,7.0,5.0,2.0,63.0,22.0
397,328,Miyako,1470 Fillmore St,San Francisco,CA,94115,37.783017,-122.432584,POINT (-122.432584 37.783017),1.415554e+10,...,Routine - Unscheduled,328_20161122_103149,Wiping cloths not clean or properly stored or ...,Low Risk,41.0,9.0,11.0,15.0,29490.0,39.0


In [8]:
df_risk = sf_df.groupby('risk_category', axis=0).count()
df_risk.head(10)

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,...,inspection_score,inspection_type,violation_id,violation_description,Neighborhoods (old),Police Districts,Supervisor Districts,Fire Prevention Districts,Zip Codes,Analysis Neighborhoods
risk_category,,,,,,,,,,,,,,,,,,,,,
High Risk,735,735,735,735,735,735,735,735,735,735,...,735,735,735,735,735,735,735,735,735,735
Low Risk,2538,2538,2538,2538,2538,2538,2538,2538,2538,2538,...,2538,2538,2538,2538,2538,2538,2538,2531,2538,2538
Moderate Risk,1942,1942,1942,1942,1942,1942,1942,1942,1942,1942,...,1942,1942,1942,1942,1942,1942,1942,1942,1942,1942
